# 03.05 - Hierarchical Indexing

Beside one-dimensional (<code>Series</code>) and two-dimensional (<code>Dataframe</code>) data, Pandas allows us to store also multidimensional data, i.e. data that is indexed by more than two keys.

**Hierarchical indexing** or multi-indexing can do just this, incorporating multiple index levels within a single index.

In [1]:
import pandas as pd
import numpy as np

### A Multiply Indexed Series

Let's start by examining how to represent two-dimensional data within a one-dimensional Series:

In [2]:
index = [('California', 2000), ('California', 2010),
         ('New York', 2000), ('New York', 2010),
         ('Texas', 2000), ('Texas', 2010)]
populations = [33871648, 37253956,
               18976457, 19378102,
               20851820, 25145561]
pop = pd.Series(populations, index=index)
pop

(California, 2000)    33871648
(California, 2010)    37253956
(New York, 2000)      18976457
(New York, 2010)      19378102
(Texas, 2000)         20851820
(Texas, 2010)         25145561
dtype: int64

### Pandas MultiIndex

In [3]:
index = pd.MultiIndex.from_tuples(index)
index

MultiIndex(levels=[['California', 'New York', 'Texas'], [2000, 2010]],
           labels=[[0, 0, 1, 1, 2, 2], [0, 1, 0, 1, 0, 1]])

If we re-index our series with this MultiIndex, we see the hierarchical representation of the data:

In [4]:
pop = pop.reindex(index)
pop

California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
dtype: int64

Now we can simply use Pandas slicing notation to access the data:

In [5]:
pop[:, 2010]

California    37253956
New York      19378102
Texas         25145561
dtype: int64

### MultiIndex as extra dimension

The <code>unstack()</code> method will quickly convert a multiply indexed Series into a conventionally indexed DataFrame:

In [6]:
pop_df = pop.unstack()
pop_df

,2000,2010
California,33871648,37253956
New York,18976457,19378102
Texas,20851820,25145561


As you would expect, <code>stack</code> does exactly the opposite:

In [7]:
pop_df.stack()

California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
dtype: int64

Then why bother with hierarchical indexing? Well, it turns out that multi-indexing is particularly helpful to add extra dimensions to our data, which is as easy as adding another column to our Dataframe:

In [8]:
pop_df = pd.DataFrame({'total': pop,
                       'under18': [9267089, 9284094,
                                   4687374, 4318033,
                                   5906301, 6879014]})
pop_df

total  under18
California 2000  33871648  9267089
           2010  37253956  9284094
New York   2000  18976457  4687374
           2010  19378102  4318033
Texas      2000  20851820  5906301
           2010  25145561  6879014

Additionally, ufuncs work with hierarchical indexing as well:

In [9]:
f_u18 = pop_df['under18'] / pop_df['total']
f_u18.unstack()

,2000,2010
California,0.273594,0.249211
New York,0.247010,0.222831
Texas,0.283251,0.273568


### Methods of MultiIndex Creation

The simplest way is to pass a list of two or more index arrays to the constructor:

In [10]:
df = pd.DataFrame(np.random.rand(4, 2),
                  index=[['a', 'a', 'b', 'b'], [1, 2, 1, 2]],
                  columns=['data1', 'data2'])
df

data1     data2
a 1  0.305225  0.569980
  2  0.625785  0.772163
b 1  0.550044  0.817318
  2  0.665885  0.995034

Additionally, for appropriately formatted dicts, Pandas will proceed with hierarchical indexing by default:

In [11]:
data = {('California', 2000): 33871648,
        ('California', 2010): 37253956,
        ('Texas', 2000): 20851820,
        ('Texas', 2010): 25145561,
        ('New York', 2000): 18976457,
        ('New York', 2010): 19378102}
pd.Series(data)

California  2000    33871648
            2010    37253956
Texas       2000    20851820
            2010    25145561
New York    2000    18976457
            2010    19378102
dtype: int64

### Explicit MultiIndex constructors

There are several ways to construct the <code>MultiIndex</code> explicitely:

In [12]:
# index values separate
pd.MultiIndex.from_arrays([['a', 'a', 'b', 'b'], [1, 2, 1, 2]])

MultiIndex(levels=[['a', 'b'], [1, 2]],
           labels=[[0, 0, 1, 1], [0, 1, 0, 1]])

In [13]:
# value, index tuples
pd.MultiIndex.from_tuples([('a', 1), ('a', 2), ('b', 1), ('b', 2)])

MultiIndex(levels=[['a', 'b'], [1, 2]],
           labels=[[0, 0, 1, 1], [0, 1, 0, 1]])

In [14]:
# cartesian product of single indices:
pd.MultiIndex.from_product([['a', 'b'], [1, 2]])

MultiIndex(levels=[['a', 'b'], [1, 2]],
           labels=[[0, 0, 1, 1], [0, 1, 0, 1]])

Or we can explicit both levels and labels:

In [15]:
pd.MultiIndex(levels=[['a', 'b'], [1, 2]],
              labels=[[0, 0, 1, 1], [0, 1, 0, 1]])

MultiIndex(levels=[['a', 'b'], [1, 2]],
           labels=[[0, 0, 1, 1], [0, 1, 0, 1]])

### MultiIndex level names

We can also name the levels of the MultiIndex using the argument <code>names</code>:

In [16]:
pop.index.names = ['state', 'year']
pop

state       year
California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
dtype: int64

### MultiIndex for columns

Rows and cols being symmetric, we can obviously have multiple levels for columns as well: 

In [36]:
# hierarchical indices and columns
index = pd.MultiIndex.from_product([[2013, 2014], [1, 2]],
                                   names=['year', 'visit'])
columns = pd.MultiIndex.from_product([['Bob', 'Guido', 'Sue'], ['HR', 'Temp']],
                                     names=['subject', 'type'])

# mock some data
data = np.round(np.random.randn(4, 6), 1)
data[:, ::2] *= 10
data += 37

# create the DataFrame
health_data = pd.DataFrame(data, index=index, columns=columns)
health_data

subject      Bob       Guido         Sue      
type          HR  Temp    HR  Temp    HR  Temp
year visit                                    
2013 1      27.0  36.4  22.0  34.5  54.0  36.8
     2      36.0  36.1  24.0  35.8  27.0  37.3
2014 1      44.0  35.4  32.0  37.4  31.0  37.7
     2      38.0  37.2  44.0  35.9  43.0  35.3

This is very useful, as we can now access the data across four different dimension.  

Let's take for example only Guido:

In [40]:
health_data['Guido']

type          HR  Temp
year visit            
2013 1      22.0  34.5
     2      24.0  35.8
2014 1      32.0  37.4
     2      44.0  35.9

### Indexing and Slicing a MultiIndex

**Series**

In [25]:
pop

state       year
California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
dtype: int64

In [26]:
# single elements indexing with multiple terms

pop['California', 2000]

33871648

In [27]:
# partial slicing

pop.loc['California':'New York']

state       year
California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
dtype: int64

Previously seen types of indexing/selection work as well, e.g. Boolean masks:

In [28]:
pop[pop > 22000000]

state       year
California  2000    33871648
            2010    37253956
Texas       2010    25145561
dtype: int64

And fancy indexing:

In [30]:
pop[['California', 'Texas']]

state       year
California  2000    33871648
            2010    37253956
Texas       2000    20851820
            2010    25145561
dtype: int64

**Dataframes**

In [31]:
health_data

subject      Bob       Guido         Sue      
type          HR  Temp    HR  Temp    HR  Temp
year visit                                    
2013 1      23.0  37.7  33.0  37.7  34.0  36.7
     2      27.0  37.3  37.0  36.2  32.0  37.3
2014 1      28.0  35.6  27.0  36.1  47.0  39.1
     2      41.0  39.0  50.0  37.6  30.0  37.0

We can easily retrieve databased on multiple indexes on DataFrames as well. For example, Sue's temperature:

In [41]:
health_data['Sue', 'Temp']

year  visit
2013  1        36.8
      2        37.3
2014  1        37.7
      2        35.3
Name: (Sue, Temp), dtype: float64